In [ ]:
from __future__ import print_function, division
import numpy as np
import scipy as sp

import mat_neuron._model as mat
from dstrf import strf, mle

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

In [ ]:
# model parameters: (ω, α1, α2, τ1, τ2, tref)
# matparams = np.asarray([7, 100, 2, 10, 200, 2], dtype='d')
matparams = np.asarray([8, -2, 1, 10, 200, 2], dtype='d')
model_dt = 0.5

# convolution kernel
from dstrf import filters
stim_dt = 10.0
kscale = 1.0
ntau = 60
upsample = int(stim_dt / model_dt)
k1, kt = filters.gammadiff(ntau * stim_dt / 32, ntau * stim_dt / 16, 5, ntau * stim_dt, stim_dt)
k1 = k1[::-1] * kscale
plt.plot(-kt[::-1], k1)


In [ ]:
def filter_stimulus(S, k1):
    return np.correlate(S, k1, mode="full")[:S.size]

def predict_spikes(V, params, dt, upsample):
    omega, a1, a2, t1, t2, tref = params
    return mat.predict_poisson(V - omega, (a1, a2), (t1, t2), tref, 
                               dt, upsample)

In [ ]:
# data parameters
duration = 100000
n_samples = int(duration / model_dt)
n_assim = 10
n_test = 5

# generate data to fit
np.random.seed(1)
mat.random_seed(1)
data = []
stim = np.random.randn(n_samples // upsample)
#stim[:100] = 0
        
V = filter_stimulus(stim, k1)
for i in range(n_assim + n_test):
    spikes = predict_spikes(V, matparams, model_dt, upsample)
    H = mat.adaptation(spikes, matparams[3:5], model_dt)
    z = np.nonzero(spikes)[0]
    d = {"H": H,
         "duration": duration,
         "spike_t": z, 
         "spike_v": spikes,
        }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:n_assim]
test_data = data[n_assim:]

In [ ]:
t_stim = np.linspace(0, duration, stim.size)
plt.subplot(311).plot(t_stim, stim)
plt.subplot(312).plot(t_stim, V)
ax2 = plt.subplot(313)
for i, d in enumerate(data):
    ax2.vlines(d["spike_t"] * model_dt, i, i + 0.5)
for ax in plt.gcf().axes:
    ax.set_xlim(0, 8000)
print("spikes: {}; rate: {} / dt".format(np.mean([d["spike_t"].size for d in data]), 
                                         np.mean([d["spike_t"].size / d["duration"] for d in data])))

### Estimation

In [ ]:
from theano import config
import scipy.optimize as op

# cosine basis set
kcosbas = strf.cosbasis(ntau, 10)
kcosbas = ntau

ftype = config.floatX
# combine the trials
spikes = np.stack([d["spike_v"] for d in assim_data], axis=1).astype(ftype)
# spikes in the exponential basis set
X_spikes = np.stack([d["H"] for d in data], axis=2).astype(ftype)
# generate design matrix for stimulus
X_stim = strf.lagged_matrix(stim, kcosbas)
# "correct" strf from mean rate
stx = np.dot(X_stim.T, V) / V.size / stim.var()
# initial guess of strf from sta
sta = strf.correlate(X_stim, spikes)

plt.plot(k1)
plt.plot(strf.from_basis(stx, kcosbas))
plt.plot(strf.from_basis(sta, kcosbas))

In [ ]:
from theano import function, config, shared, sparse, gradient
import theano.tensor as T
from theano.tensor import nnet
import scipy.sparse as sps

# the nonlinearity:
nlin = T.exp

if X_spikes.ndim == 2:
    spike_design = np.expand_dims(X_spikes, 2)
if spikes.ndim == 1:
    spikes = np.expand_dims(spikes, 1)

nframes, nk = X_stim.shape
nbins, nalpha, ntrials = X_spikes.shape
upsample = int(stim_dt / model_dt)
# make an interpolation matrix
interp = sps.kron(sps.eye(nframes),
                  np.ones((upsample, 1), dtype=config.floatX),
                  format='csc')

# load the data into theano.shared structures
M = shared(interp)
dt = shared(model_dt)
Xstim = shared(X_stim)
Xspke = shared(np.rollaxis(X_spikes, 2))
spkx, spky = map(shared, spikes.nonzero())

# split out the parameter vector
w = T.vector('w')
dc = w[0]
h = w[1:(nalpha+1)]
k = w[(nalpha+1):]
Vx = T.dot(Xstim, k)
# Vx has to be promoted to a matrix for structured_dot to work
Vi = sparse.structured_dot(M, T.shape_padright(Vx))
H = T.dot(Xspke, h).T
mu = Vi - H - dc
lmb = nlin(mu)
ll = lmb.sum() * dt - T.log(lmb[spkx, spky]).sum()
dL = T.grad(ll, w)
# arbitrary vector for hessian-vector product
v = T.vector('v')
ddLv = T.grad(T.sum(dL * v), w)

fV = function([w], Vx)
fH = function([w], H)
fL = function([w], ll)
fgrad = function([w], dL)
fhess = function([w, v], ddLv)

In [ ]:
# initial likelihood
meanrate = spikes.sum(0).mean() / nbins
w0 = np.r_[np.exp(meanrate), 0, 0, np.zeros_like(sta)]
fL(w0)

In [ ]:
w_ml = op.fmin_ncg(fL, w0, fgrad, fhess_p=fhess, avextol=1e-6, maxiter=200)

In [ ]:
print(matparams[:3])
print(w_ml[:3])
plt.plot(k1)
plt.plot(strf.from_basis(w_ml[3:], kcosbas))

In [ ]:
# posterior predictive distribution
for j, d in enumerate(test_data):
    plt.vlines(d["spike_t"], j, j + 0.5, 'r')

mparamp = matparams.copy()    
for i in range(n_test):
    V = fV(w_ml)
    mparamp[:3] = w_ml[:3]
    S = predict_spikes(V, mparamp, model_dt, upsample)
    spk_t = S.nonzero()[0]
    plt.vlines(spk_t, i + j + 1, i + j + 1.5)

plt.xlim(0, 10000)